<em><sub>This page is available as an executable or viewable <strong>Jupyter Notebook</strong></sub></em>
<br/><br/>
<a href="https://mybinder.org/v2/gh/avan1235/KotlinDL/notebooks?filepath=docs%2Floading_trained_model_for_inference.ipynb"
   target="_parent">
   <img align="left"
        src="https://mybinder.org/badge_logo.svg"
        height="20">
</a>
<a href="https://nbviewer.jupyter.org/github/avan1235/KotlinDL/blob/notebooks/docs/loading_trained_model_for_inference.ipynb"
   target="_parent">
   <img align="right"
        src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg"
        height="20">
</a>
<br/><br/>

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-api:0.2.0")

In the previous tutorials, we [created](create_your_first_nn.ipynb), [trained, and saved](training_a_model.ipynb) a deep learning model.
Now let's look at how we can load and use that model to generate predictions on new, previously unseen data.

To do this, we will quickly reuse the code from the previous tutorials to provide the actual files with model state. Let's see how we can use the gained experience to briefly create, train and save the sample model.

In [2]:
import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.core.WritingMode
import org.jetbrains.kotlinx.dl.api.core.layer.core.Input
import org.jetbrains.kotlinx.dl.api.core.layer.reshaping.Flatten
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.dataset.fashionMnist
import java.io.File


val modelLocation = File("src/model/my_first_model")

// extract the train and test data samples with labels
val (train, test) = fashionMnist()

// define our model
Sequential.of(
    Input(28, 28, 1),
    Flatten(),
    Dense(300),
    Dense(100),
    Dense(10)
).use { model ->
    // compile model for usage
    model.compile(
        optimizer = Adam(),
        loss = Losses.SOFT_MAX_CROSS_ENTROPY_WITH_LOGITS,
        metric = Metrics.ACCURACY
    )
    // train compiled model
    model.fit(
        dataset = train,
        epochs = 10,
        batchSize = 100
    )
    // save model weights to local file
    model.save(modelLocation, writingMode = WritingMode.OVERRIDE)
}

Extracting 60000 images of 28x28 from /workspace/cache/datasets/fashionmnist/train-images-idx3-ubyte.gz
Extracting 60000 labels from /workspace/cache/datasets/fashionmnist/train-labels-idx1-ubyte.gz
Extracting 10000 images of 28x28 from /workspace/cache/datasets/fashionmnist/t10k-images-idx3-ubyte.gz
Extracting 10000 labels from /workspace/cache/datasets/fashionmnist/t10k-labels-idx1-ubyte.gz


For illustration purposes, and to simplify the data processing in this tutorial, 
we'll use the test data to make a prediction example of the loaded model on the data
the model has not been trained on.

The example images in the test data have the same size and format as the ones the model has been trained on, 
so we do not need to do any additional preprocessing. 
However, if you are going to train the model on your own data, 
make sure to use the image preprocessing before using an image for inference to get
exactly the same input structure for model as it was trained on. 

To load the model simply use the path to it, tell it how incoming images should be reshaped (if needed), and call the
 `predict` method on them.

In [5]:
import org.jetbrains.kotlinx.dl.api.inference.InferenceModel

val labelsMap = mapOf(
        0 to "T-shirt/top",
        1 to "Trousers",
        2 to "Pullover",
        3 to "Dress",
        4 to "Coat",
        5 to "Sandals",
        6 to "Shirt",
        7 to "Sneakers",
        8 to "Bag",
        9 to "Ankle boots"
)

InferenceModel.load(modelLocation).use { model ->
                                        
    model.reshape(28, 28, 1)

    val prediction = model.predict(test.getX(0))
    val actualLabel = test.getY(0)
    
    println("Predicted label is: $prediction.")
    println("This corresponds to class ${labelsMap[prediction]}.")
    println("Actual label is: $actualLabel.")
}

Predicted label is: 9.
This corresponds to class Ankle boots.
Actual label is: 9.0.


This shows how short and easy can be usage of an already trained model comparing to the code
needed to train the model that we presented for the completeness of this example.

Congratulations! You have learned how to create, train, and use your first neural network model!
Now you may be interested in [loading model from Keras](importing_keras_model.ipynb) or getting
familiar with [transfer learning in KotlinDL](transfer_learning.ipynb).